## 구글 드라이브 마운트
colab 런타임을 본인의 구글 드라이브에 마운트시킨 후, 적절한 경로를 이동해 줍니다.

In [1]:
import os
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

# 경로로 이동해 주세요.
path = '/content/drive/My Drive/bert_classification_kor/eojeol'
os.chdir(path)

Mounted at /content/drive


In [2]:
os.listdir()

['inference.py',
 'bert_to_keras.py',
 'eval_bert_finetuning.py',
 'utils.py',
 'train_bert_finetuning.py',
 'prepare_data.py',
 'modeling.py',
 '.DS_Store',
 'models',
 'readers',
 'vectorizers',
 'layers',
 '004_bert_eojeol_tensorflow',
 'hello_BERT',
 '__pycache__',
 'data',
 'KorBERT_classification.ipynb']

## 환경 설정
KorBERT를 활용할 수 있는 환경을 설정해 줍니다. 우선 아래와 같은 모듈들을 설치해 줍니다.
  - tensorflow-gpu==1.15
  - tensorflow-hub==0.7.0


그 후, 사용 가능한 gpu를 환경변수 CUDA_VISIBLE_DEVICES 변수에 설정해 줍니다.

In [ ]:
!pip install tensorflow-gpu==1.15
!pip install tensorflow-hub==0.7.0

사용 가능한 gpu를 확인해 줍니다. 만일 런타임 유형 변경에서 가속기로 gpu를 설정하지 않는다면 다음과 같은 에러 메시지가 나올 것입니다.

`NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.`

이럴 경우 런타임 유형을 변경한 후 다시 실행합니다.

In [4]:
!export CUDA_VISIBLE_DEVICES=0 # 0번 gpu를 쓰겠다
!nvidia-smi 

Fri Oct 29 08:44:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# checkpoint -> keras layer
ETRI의 KorBERT를 keras model로 변환해 줍니다. 이는 `bert_to_keras.py`로 가능합니다.

In [ ]:
# bert_module로 변환해 줬으면 이 과정은 생략하셔도 됩니다.
# kobert -> 케라스 레이어로 변환
!python bert_to_keras.py    # bert_to_keras.py파일을 python으로 실행하는 코드

In [6]:
# 'bert-module'이라는 경로가 추가적으로 생성된 것을 확인할 수 있습니다.
os.listdir()

['inference.py',
 'bert_to_keras.py',
 'eval_bert_finetuning.py',
 'utils.py',
 'train_bert_finetuning.py',
 'prepare_data.py',
 'modeling.py',
 '.DS_Store',
 'models',
 'readers',
 'vectorizers',
 'layers',
 '004_bert_eojeol_tensorflow',
 'hello_BERT',
 '__pycache__',
 'data',
 'KorBERT_classification.ipynb',
 'bert-module']

## 데이터 전처리
KorBERT가 Fine-Tune을 할 수 있게끔 데이터를 준비해 주어야 합니다. 준비해야 하는 데이터 파일은 tab(\t)을 기준으로 label과 input sequence가 나뉘어 있어야 합니다.

`prepare_data.py`를 실행할 경우 data -> label, seq.in 으로 나뉘게 되는데, 이떄 seq.in은 KorBERT가 pre-train된 대로 tokenization 과정을 거치게 됩니다.

`prepare_data.py`의 경우 mandatory한 인자로 --input(-i)와 --output(-o)를 갖습니다.

`python prepare_data.py --input ${input_data} --output ${processed_data_path}`


In [7]:
# !python prepare_data.py --input starbucks_data/train/train_data.txt --output starbucks_data/train/

"""
prepare_data.py

options:
--input  [-i] [필수] txt File        # label과 text가 \t로 구분된 txt파일
--output [-o] [필수] Path to data    # 변환된 파일을 저장할 경로
"""

!python prepare_data.py --input data/sample_data.txt --output sample_data/


KorBERT의 tokenization 방식을 알기 위해서는 다음 코드와

In [30]:
from vectorizers import tokenizationK as tk


sample_sentence = '와 이 커피 진짜 맛있다.'
### 소스코드를 참조하여 알맞게 채워보세요 ###
vocab_file_path = '/content/drive/MyDrive/bert_classification_kor/eojeol/004_bert_eojeol_tensorflow/vocab.korean.rawtext.list'

tokenizer = tk.FullTokenizer(vocab_file_path)

tokens = ""

for word in sample_sentence.split():
    word_tokens = " ".join(tokenizer.tokenize(word))
    tokens += word_tokens + " "

In [31]:
tokens

'와_ 이_ 커 피_ 진짜_ 맛 있다 ._ '

## Fine-Tuning 훈련

준비된 데이터를 바탕으로 KorBERT fine-tuning을 실시해 줍니다.

`train_bert_finetuning.py`를 실행하며, mandatory 인자로는 --train(-t) 와 --save(-s)를 받습니다.

`python train_bert_finetuning.py --train ${processed_data_path} --save ${saved_model}`

In [15]:
""" 
train_bert_finetuning.py    # 거의 model.fit이라고 생각하면 됨

option:
--train     [-t]    [필수]              Path to training data                   # training_data가 있는 폴더 경로
--val       [-v]    [default = False]   Path to validation data                 # validation_data가 있는 폴더 경로
--save      [-s]    [필수]              Folder path to save the trained model   # model을 저장할 폴더 경로
--epochs    [-e]    [defalut = 5]       Numbers of epochs                       
--batch     [-bs]   [default = 64]      Batch size                              
--type      [-tp]   [default = bert]    'bert' or 'albert'                      # bert나 albert 중 어떤것을 사용할 것인지

"""

!python train_bert_finetuning.py --train sample_data/ --save train_model



2021-10-29 08:48:36.678994: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-10-29 08:48:36.684549: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-10-29 08:48:36.685023: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x558333e6ea00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-10-29 08:48:36.685064: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-10-29 08:48:36.686907: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-10-29 08:48:36.818039: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-29 08:48:36.818961:

## EVALUATION

In [37]:
# tensorflow의 버전이 맞지않아 추가적인 모듈 다운로딩 필요
!pip install h5py==2.10.0

     |████████████████████████████████| 2.9 MB 5.5 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires gast==0.4.0, but you have gast 0.2.2 which is incompatible.
tensorflow 2.6.0 requires h5py~=3.1.0, but you have h5py 2.10.0 which is incompatible.
tensorflow 2.6.0 requires tensorboard~=2.6, but you have tensorboard 1.15.0 which is incompatible.
tensorflow 2.6.0 requires tensorflow-estimator~=2.6, but you have tensorflow-estimator 1.15.1 which is incompatible.


In [39]:
"""
eval_bert_finetuning.py  # model.evaluate 생각하면 될 듯

options:

--model [-m]    [필수]              Path to Bert            # 위의 train_bert_finetuning.py 에서 생성한 모델폴더
--data  [-d]    [필수]              Path to data            # test_data가 들어있는 폴더
--type  [-tp]   [defalut = 'bert']  'bert' or 'albert'      

"""

!python eval_bert_finetuning.py -m train_model -d ./sample_data --type bert



2021-10-29 09:14:56.959578: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-10-29 09:14:56.966104: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-10-29 09:14:56.966689: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x555c7a988a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-10-29 09:14:56.966729: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-10-29 09:14:56.969246: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-10-29 09:14:56.973911: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-29 09:14:56.973986: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:16